# Information Retrieval using word2vec based Vector Space Model
### Ananya Uppal (PES1UG19CS058), Anchal Sharma (PES1UG19CS059), Maitreyi P (PES1UG19CS254)

Dataset being used : https://microsoft.github.io/msmarco/TREC-Deep-Learning-2019
The document ranking 

In [ ]:
!pip install dask[complete]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.tsv.gz
# !wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-queries.tsv.gz
# !wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-top100.gz
# !gzip -d msmarco-docs.tsv.gz
# !gzip -d msmarco-doctrain-queries.tsv.gz
# !gzip -d msmarco-doctrain-top100.gz

In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

Load File Containing Queries

In [ ]:
queries_train=pd.read_table('/content/drive/MyDrive/Colab Notebooks/AIR/queries-doctrain.tsv',header=None)
#queries_train=pd.read_table('/content/drive/MyDrive/queries-doctrain.tsv',header=None)
queries_train.columns=['qid','query']
print('Shape=>',queries_train.shape)
queries_train.head()

Shape=> (367013, 2)


,qid,query
0,1185869,)what was the immediate impact of the success ...
1,1185868,_________ justice is designed to repair the ha...
2,1183785,elegxo meaning
3,645590,what does physical medicine do
4,186154,feeding rice cereal how many times per day


In [ ]:
# Choose a sample set of 2000 queries
queries=queries_train.sample(n=2000,random_state=42).reset_index(drop=True)
print('Shape=>',queries.shape)
queries.head()

Shape=> (2000, 2)


,qid,query
0,687888,what is a jpe
1,480210,price for asphalt driveway
2,591004,what causes pressure skin bruising
3,260536,how long drive from flagstaff to grand canyon
4,39422,average number of bowel movements per day for ...


In [ ]:
# create two sets out of this, each of 1000 queries. One will be the training set, and other will be the testing set
# Creating Training Set of Queries
training_queries=queries.iloc[:1000]
print('Shape=>',training_queries.shape)
training_queries.head()

Shape=> (1000, 2)


,qid,query
0,687888,what is a jpe
1,480210,price for asphalt driveway
2,591004,what causes pressure skin bruising
3,260536,how long drive from flagstaff to grand canyon
4,39422,average number of bowel movements per day for ...


In [ ]:
# Creating Testing Set of Queries
testing_queries=queries.iloc[1000:]
print('Shape=>',testing_queries.shape)
testing_queries.head()

Shape=> (1000, 2)


,qid,query
1000,807599,what is the axis mundi?
1001,990945,where is pratt kansas
1002,48210,backordered definition
1003,894254,what show did simon baker play in
1004,165579,does dna replication in mitosis and meiosis


In [ ]:
# load data containing top 100 documents for each query
train_top100=pd.read_table('/content/drive/MyDrive/Colab Notebooks/AIR/msmarco-doctrain-top100.tsv',delimiter=' ',header=None)
#train_top100=pd.read_table('/content/drive/MyDrive/msmarco-doctrain-top100.tsv',delimiter=' ',header=None)
#train_top100=pd.read_table('msmarco-doctrain-top100',delimiter=' ',header=None)
train_top100.columns=['qid','Q0','docid','rank','score','runstring']
print('Shape=>',train_top100.shape)
train_top100.head()

Shape=> (36701116, 6)


,qid,Q0,docid,rank,score,runstring
0,1185869,Q0,D59221,1,-4.80433,IndriQueryLikelihood
1,1185869,Q0,D59220,2,-4.92127,IndriQueryLikelihood
2,1185869,Q0,D2192591,3,-5.05215,IndriQueryLikelihood
3,1185869,Q0,D2777518,4,-5.05486,IndriQueryLikelihood
4,1185869,Q0,D2371978,5,-5.07048,IndriQueryLikelihood


In [ ]:
# Reducing train_top100 for training
training_ranked100=train_top100[train_top100['qid'].isin(training_queries['qid'].unique())].reset_index(drop=True)
print('Shape=>',training_ranked100.shape)
training_ranked100.head()

Shape=> (100000, 6)


,qid,Q0,docid,rank,score,runstring
0,310290,Q0,D579750,1,-5.11498,IndriQueryLikelihood
1,310290,Q0,D579754,2,-5.57703,IndriQueryLikelihood
2,310290,Q0,D2380815,3,-5.84852,IndriQueryLikelihood
3,310290,Q0,D822566,4,-5.95002,IndriQueryLikelihood
4,310290,Q0,D2249695,5,-6.08326,IndriQueryLikelihood


In [ ]:
# Reducing train_top100 for testing
testing_ranked100=train_top100[train_top100['qid'].isin(testing_queries['qid'].unique())].reset_index(drop=True)
print('Shape=>',testing_ranked100.shape)
testing_ranked100.head()



Shape=> (100000, 6)


,qid,Q0,docid,rank,score,runstring
0,1164761,Q0,D3261512,1,-4.92320,IndriQueryLikelihood
1,1164761,Q0,D1529569,2,-5.01292,IndriQueryLikelihood
2,1164761,Q0,D3444265,3,-5.03616,IndriQueryLikelihood
3,1164761,Q0,D1313045,4,-5.09482,IndriQueryLikelihood
4,1164761,Q0,D1058999,5,-5.18936,IndriQueryLikelihood


In [ ]:
#Reducing it further and creating a ground truth for evaluation

# Labelling Top 10 as 1 and last 10 as 0
rel=list(range(1,11))
nonrel=list(range(91,101))
training_ranked100['rel']=training_ranked100['rank'].apply(lambda x: 1 if x in rel else ( 0 if x in nonrel else np.nan))
testing_ranked100['rel']=testing_ranked100['rank'].apply(lambda x: 1 if x in rel else ( 0 if x in nonrel else np.nan))

# Result set for Training
training_result=training_ranked100.dropna()
training_result['rel']=training_result['rel'].astype(int)
print('Shape=>',training_result.shape)
print(training_result.head())


# Result set for Testing
testing_result=testing_ranked100.dropna()
testing_result['rel']=testing_result['rel'].astype(int)
print('Shape=>',testing_result.shape)
testing_result.head()

Shape=> (20000, 7)
      qid  Q0     docid  rank    score             runstring  rel
0  310290  Q0   D579750     1 -5.11498  IndriQueryLikelihood    1
1  310290  Q0   D579754     2 -5.57703  IndriQueryLikelihood    1
2  310290  Q0  D2380815     3 -5.84852  IndriQueryLikelihood    1
3  310290  Q0   D822566     4 -5.95002  IndriQueryLikelihood    1
4  310290  Q0  D2249695     5 -6.08326  IndriQueryLikelihood    1
Shape=> (20000, 7)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,qid,Q0,docid,rank,score,runstring,rel
0,1164761,Q0,D3261512,1,-4.92320,IndriQueryLikelihood,1
1,1164761,Q0,D1529569,2,-5.01292,IndriQueryLikelihood,1
2,1164761,Q0,D3444265,3,-5.03616,IndriQueryLikelihood,1
3,1164761,Q0,D1313045,4,-5.09482,IndriQueryLikelihood,1
4,1164761,Q0,D1058999,5,-5.18936,IndriQueryLikelihood,1


using Dask dataframes to reduce the corpus 

In [ ]:
#creating reduced dataframe
df=dd.read_table('/content/drive/MyDrive/Colab Notebooks/AIR/msmarco-docs.tsv',header=None)
#df=dd.read_table('/content/drive/MyDrive/msmarco-docs.tsv',header=None)
#df=dd.read_table('msmarco-docs.tsv',header=None)
df.columns=['docid','url','title','body']
df.head()

,docid,url,title,body
0,D1555982,https://answers.yahoo.com/question/index?qid=2...,The hot glowing surfaces of stars emit energy ...,Science & Mathematics Physics The hot glowing ...
1,D301595,http://childparenting.about.com/od/physicalemo...,Developmental Milestones and Your 8-Year-Old C...,School-Age Kids Growth & Development Developme...
2,D1359209,http://visihow.com/Check_for_Lice_Nits,Check for Lice Nits,Check for Lice Nits Edited by Mian Sheilette O...
3,D2147834,http://www.nytimes.com/2010/01/05/business/glo...,Dubai Opens a Tower to Beat All,Global Business Dubai Opens a Tower to Beat Al...
4,D1568809,http://www.realtor.com/realestateandhomes-sear...,"Coulterville, CA Real Estate & Homes for Sale","Coulterville, CA Real Estate & Homes for Sale4..."


In [ ]:
# #mapping the docid from result files with the docid in the corpus file
def create_corpus(result):
  unique_docid=result['docid'].unique()
  condition=df['docid'].isin(unique_docid)
  corpus=df[condition].reset_index(drop=True)
  corpus=corpus.drop(columns='url')
  # print('Number of Rows=>',len(corpus))
  return corpus

# creating the training corpus
training_corpus=create_corpus(training_result)
training_corpus.head()

,docid,title,body
0,D297612,"Fair Oaks, CA County Of Sacramento","Home Fair Oaks, CA County Of Sacramento Fair O..."
1,D1036761,What airport is the closest to downtown London?,Answers.com ® Wiki Answers ® Categories Travel...
2,D2025493,"York County, South Carolina Genealogy",navigation search United States South Carolina...
3,D2214523,The Natural Habitat of Wolves,"Wolves are members of the canine family, but t..."
4,D1881859,Franking Privilege Law and Legal Definition,Franking Privilege Law and Legal Definition Fr...


In [ ]:
# #creating the testing corpus
testing_corpus=create_corpus(testing_result)
testing_corpus.head()

,docid,title,body
0,D1911483,My Doctor Online The Permanente Medical Group,Abnormal Vaginal Bleeding in Midlife and Beyon...
1,D2378859,How many decibels can a human hear in?,Answers.com ® Wiki Answers ® Categories Scienc...
2,D2981241,What do you call a group of lions?,Lions Vocabulary of the English Language Word ...
3,D2337005,Peripheral Vascular Surgery-Chapter 32,165 terms alexandriamartinez19Peripheral Vascu...
4,D2078142,What are all the literary devices? List them p...,Education & Reference Homework Help What are a...


### Data Exploration

In [ ]:
#taking a sample of the corpus
temp_doc=training_corpus.sample(1)
print('Title=>',temp_doc.title.values)
print('Body:\n',temp_doc.body.values)

Title=> dask.array<values, shape=(nan,), dtype=object, chunksize=(nan,), chunktype=numpy.ndarray>
Body:
 dask.array<values, shape=(nan,), dtype=object, chunksize=(nan,), chunktype=numpy.ndarray>


/usr/local/lib/python3.7/dist-packages/dask/dataframe/core.py:1274: UserWarning: sample does not support the number of sampled items parameter, 'n'. Please use the 'frac' parameter instead.
  warnings.warn(msg)


In [ ]:
#sampling the queries
for i,v in enumerate(training_queries['query'].sample(5)):
    print(i,'=>',v)

0 => when did tim tebow win the heisman trophy
1 => dow jones industrial average etf
2 => is nexplanon an implantable subdermal
3 => do bed bugs bite
4 => what are the swiss known for


Preprocessing the text

Documents:
* Lowercase the text
* Expand Contractions
* Clean the text
* Remove Stopwords
* Lemmatize words


Queries:
* Lowercase the text
* Expand Contractions
* Clean the text

In [ ]:
#DOCUMENTS
import re

# 1. Lowercasing the text
training_corpus['cleaned']=training_corpus['body'].apply(lambda x:x.lower())
testing_corpus['cleaned']=testing_corpus['body'].apply(lambda x:x.lower())


/usr/local/lib/python3.7/dist-packages/dask/dataframe/core.py:3073: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('body', 'object'))

  warnings.warn(meta_warning(meta))


In [ ]:
# 2. Expanding Contarctions
# Dictionary of english Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not","can't": "can not","can't've": "cannot have",
"'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
"didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
"hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
"he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
"i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
"isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
"let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
"mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
"needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
"shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will",
"she'll've": "she will have","should've": "should have","shouldn't": "should not",
"shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
"there'd": "there would","there'd've": "there would have",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
"they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
"we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
"weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
"what've": "what have","when've": "when have","where'd": "where did",
"where've": "where have","who'll": "who will","who'll've": "who will have","who've": "who have",
"why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
"would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
"you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
"you're": "you are","you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

training_corpus['cleaned']=training_corpus['cleaned'].apply(lambda x:expand_contractions(x))
testing_corpus['cleaned']=testing_corpus['cleaned'].apply(lambda x:expand_contractions(x))

/usr/local/lib/python3.7/dist-packages/dask/dataframe/core.py:3073: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('cleaned', 'object'))

  warnings.warn(meta_warning(meta))


In [ ]:
# 3. Cleaning the text
def clean_text(text):
    text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^a-z]',' ',text)
    return text
 
training_corpus['cleaned']=training_corpus['cleaned'].apply(lambda x: clean_text(x))
testing_corpus['cleaned']=testing_corpus['cleaned'].apply(lambda x: clean_text(x))
# Removing extra spaces
training_corpus['cleaned']=training_corpus['cleaned'].apply(lambda x: re.sub(' +',' ',x))
testing_corpus['cleaned']=testing_corpus['cleaned'].apply(lambda x: re.sub(' +',' ',x))

/usr/local/lib/python3.7/dist-packages/dask/dataframe/core.py:3073: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('cleaned', 'object'))

  warnings.warn(meta_warning(meta))


In [ ]:
# 4. Removing Stopwords and Lemmatizing using Spacy
import spacy

nlp = spacy.load('en_core_web_sm',disable=['ner','parser'])
nlp.max_length=5000000

training_corpus['lemmatized']=training_corpus['cleaned'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))
testing_corpus['lemmatized']=testing_corpus['cleaned'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

/usr/local/lib/python3.7/dist-packages/dask/dataframe/core.py:3073: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('cleaned', 'object'))

  warnings.warn(meta_warning(meta))


In [ ]:
#QUERIES
# 1. Lowercasing
training_queries['cleaned']=training_queries['query'].apply(lambda x:x.lower())
testing_queries['cleaned']=testing_queries['query'].apply(lambda x:x.lower())

# 2. Expanding contractions
training_queries['cleaned']=training_queries['cleaned'].apply(lambda x:expand_contractions(x))
testing_queries['cleaned']=testing_queries['cleaned'].apply(lambda x:expand_contractions(x))

# 3. Cleaning queries using RegEx
training_queries['cleaned']=training_queries['cleaned'].apply(lambda x: clean_text(x))
testing_queries['cleaned']=testing_queries['cleaned'].apply(lambda x: clean_text(x))
training_queries['cleaned']=training_queries['cleaned'].apply(lambda x: re.sub(' +',' ',x))
testing_queries['cleaned']=testing_queries['cleaned'].apply(lambda x: re.sub(' +',' ',x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

### Creating Vectors
Training Word2Vec model and generating vectors for docs and queries

In [ ]:
# Combining corpus and queries for training
#combined_training=dd.concat([training_corpus.rename(columns={'lemmatized':'text'})['text'],training_queries.rename(columns={'cleaned':'text'})['text']]).sample(frac=1).reset_index(drop=True)


In [ ]:
#loading the combined training df
#combined_training=pd.read_csv('/content/drive/MyDrive/combined_training.csv',header=None)
combined_training=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIR/combined_training.csv',header=None)

In [ ]:
#4combined_training.head()

In [ ]:
#deleting training dataframe
del[training_corpus] 
del[training_queries]
del[queries_train]
del[train_top100] 

training_corpus = None 
training_queries = None
queries_train = None
train_top100 = None

In [ ]:
#combined_training = combined_training.compute()

In [ ]:
#training the W2V model using Gensim
from gensim.models import Word2Vec

# Creating data for the model training
train_data=[]

for i in combined_training[1]:
    #train_data.append(i.split())
    train_data.append(i.split())

In [ ]:
# Training 
w2v_model = Word2Vec(train_data, size=300, min_count=2,window=5, sg=1,workers=4)

In [ ]:
w2v_model.save("word2vec.model")

In [ ]:
# 1w2v_model = Word2Vec.load("/content/word2vec.model")

In [ ]:
# Vocab size
print('Vocabulary size:', len(w2v_model.wv.vocab))

Vocabulary size: 198281


In [ ]:
#saving combined_training
# from google.colab import files

# combined_training.to_csv('saving combined_training.csv', encoding = 'utf-8-sig') 
# files.download('saving combined_training.csv')

In [ ]:
#deleting combined_training from memory
del[combined_training]

In [ ]:
import gc
gc.collect()

146

In [ ]:
testing_corpus.head()

,docid,title,body,cleaned,lemmatized
0,D1911483,My Doctor Online The Permanente Medical Group,Abnormal Vaginal Bleeding in Midlife and Beyon...,abnormal vaginal bleeding in midlife and beyon...,abnormal vaginal bleeding midlife overview wom...
1,D2378859,How many decibels can a human hear in?,Answers.com ® Wiki Answers ® Categories Scienc...,answers com wiki answers categories science ph...,answer com wiki answer category science physic...
2,D2981241,What do you call a group of lions?,Lions Vocabulary of the English Language Word ...,lions vocabulary of the english language word ...,lion vocabulary english language word definiti...
3,D2337005,Peripheral Vascular Surgery-Chapter 32,165 terms alexandriamartinez19Peripheral Vascu...,terms vascular surgery chapter flashcards wri...,term vascular surgery chapter flashcard writ...
4,D2078142,What are all the literary devices? List them p...,Education & Reference Homework Help What are a...,education reference homework help what are all...,education reference homework help literary dev...


In [ ]:
testing_corpus.drop('body', axis = 1)
testing_corpus.drop('title', axis =1)
testing_corpus.drop('cleaned', axis = 1)

,docid,title,body,lemmatized
npartitions=358,,,,
,object,object,object,object
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [ ]:
testing_corpus.head()

,docid,title,body,cleaned,lemmatized
0,D1911483,My Doctor Online The Permanente Medical Group,Abnormal Vaginal Bleeding in Midlife and Beyon...,abnormal vaginal bleeding in midlife and beyon...,abnormal vaginal bleeding midlife overview wom...
1,D2378859,How many decibels can a human hear in?,Answers.com ® Wiki Answers ® Categories Scienc...,answers com wiki answers categories science ph...,answer com wiki answer category science physic...
2,D2981241,What do you call a group of lions?,Lions Vocabulary of the English Language Word ...,lions vocabulary of the english language word ...,lion vocabulary english language word definiti...
3,D2337005,Peripheral Vascular Surgery-Chapter 32,165 terms alexandriamartinez19Peripheral Vascu...,terms vascular surgery chapter flashcards wri...,term vascular surgery chapter flashcard writ...
4,D2078142,What are all the literary devices? List them p...,Education & Reference Homework Help What are a...,education reference homework help what are all...,education reference homework help literary dev...


In [ ]:
testing_queries.head()
testing_queries.drop('query', axis = 1)

,qid,cleaned
1000,807599,what is the axis mundi
1001,990945,where is pratt kansas
1002,48210,backordered definition
1003,894254,what show did simon baker play in
1004,165579,does dna replication in mitosis and meiosis
...,...,...
1995,797493,what is social darwinism
1996,413994,is it monsoon season
1997,734522,what is contract review
1998,1137024,definition of qarm


In [ ]:
testing_queries.head()

,qid,query,cleaned
1000,807599,what is the axis mundi?,what is the axis mundi
1001,990945,where is pratt kansas,where is pratt kansas
1002,48210,backordered definition,backordered definition
1003,894254,what show did simon baker play in,what show did simon baker play in
1004,165579,does dna replication in mitosis and meiosis,does dna replication in mitosis and meiosis


In [ ]:
# Function returning vector reperesentation of a document
def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(300)
    else:
        for tok in doc_tokens:
            if tok in w2v_model.wv.vocab:
                embeddings.append(w2v_model.wv.word_vec(tok))
            else:
                embeddings.append(np.random.rand(300))
        # mean the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

In [ ]:
testing_corpus = testing_corpus.head(59)
testing_queries = testing_queries.head(500)

In [ ]:
# Getting Word2Vec Vectors for Testing Corpus and Queries
testing_corpus['vector']=testing_corpus['lemmatized'].apply(lambda x :get_embedding_w2v(x.split()))
testing_queries['vector']=testing_queries['cleaned'].apply(lambda x :get_embedding_w2v(x.split()))

## IR PIPELINE

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def ranking_ir(query):
  
  # pre-process Query
  query=query.lower()
  query=expand_contractions(query)
  query=clean_text(query)
  query=re.sub(' +',' ',query)

  # generating vector
  vector=get_embedding_w2v(query.split())

  # ranking documents
  documents=testing_corpus[['docid','title','body']]
  documents['similarity']=testing_corpus['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents.sort_values(by='similarity',ascending=False,inplace=True)
  
  return documents.head(10).reset_index(drop=True)

In [ ]:
ranking_ir('michael jordan')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,docid,title,body,similarity
0,D1910992,Mark Cuban: Adam Silver said Tony Romo contrac...,Mark Cuban: Adam Silver said Tony Romo contrac...,0.496247
1,D3510094,7 Best American Sports Teams of All Time,Jared Wickerham/Getty Images What’s the point ...,0.480316
2,D2460194,Soupy Sales,"From Wikipedia, the free encyclopedianavigatio...",0.440696
3,D1929121,Tina Turner Biography,Quick Facts Name Tina Turner Occupation Actres...,0.438860
4,D3295240,Do Prince Harry and Prince William have surnames?,Society & Culture Royalty Do Prince Harry and ...,0.417814
5,D2336712,Sellout Song Johnny Hobo Is Dead chords,Edit Add to playlist Favorite Sellout Song Joh...,0.408279
6,D1549869,ROOT Sports Southwest to Debut Nov. 17 with Pr...,ROOT Sports Southwest to Debut Nov. 17 with Pr...,0.403753
7,D1583774,How are âenlightenmentâ ideals reflected in Cl...,How are ‘enlightenment’ ideals reflected in Cl...,0.388627
8,D898283,What is a Ceilidh,"Silicon Glen, Scotland > Scottish FAQ > Scotti...",0.384978
9,D1755937,Why was the Holocaust allowed to happen?,Answers.com ® Wiki Answers ® Categories Histor...,0.383889


In [ ]:
ranking_ir('movies')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,docid,title,body,similarity
0,D2460194,Soupy Sales,"From Wikipedia, the free encyclopedianavigatio...",0.594126
1,D1929121,Tina Turner Biography,Quick Facts Name Tina Turner Occupation Actres...,0.568718
2,D1910992,Mark Cuban: Adam Silver said Tony Romo contrac...,Mark Cuban: Adam Silver said Tony Romo contrac...,0.513124
3,D1549869,ROOT Sports Southwest to Debut Nov. 17 with Pr...,ROOT Sports Southwest to Debut Nov. 17 with Pr...,0.505998
4,D898283,What is a Ceilidh,"Silicon Glen, Scotland > Scottish FAQ > Scotti...",0.480816
5,D3510094,7 Best American Sports Teams of All Time,Jared Wickerham/Getty Images What’s the point ...,0.473817
6,D2336712,Sellout Song Johnny Hobo Is Dead chords,Edit Add to playlist Favorite Sellout Song Joh...,0.461363
7,D3295240,Do Prince Harry and Prince William have surnames?,Society & Culture Royalty Do Prince Harry and ...,0.460769
8,D1921908,.,The Gospel in the Stars: Part 1By J. R. Church...,0.434074
9,D892046,Pro Street Cars for Sale,Post Ad Pro Street Cars for Sale At Hotrod Pit...,0.429015
